<a href="https://colab.research.google.com/github/wgova/time_series_trade/blob/master/clustering_tsfresh_features.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Required packages

In [ ]:
!pip install -q dtw-python
# dynamic time warping
from dtw import *
from sklearn.cluster import AgglomerativeClustering
from scipy.cluster.hierarchy import fcluster, ward, dendrogram

from scipy.cluster.vq import kmeans,vq
from math import sqrt
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler,StandardScaler,normalize
from sklearn.metrics.cluster import homogeneity_score
from sklearn import decomposition
from scipy.cluster.hierarchy import dendrogram
from sklearn.cluster import KMeans, SpectralClustering,DBSCAN 
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn.decomposition import PCA

## Functions for automating clustering

In [ ]:
# Functions for dendrograms
# given a linkage model, plog dendogram, with the colors indicated by the a cutoff point at which we define clusters
#https://scikit-learn.org/stable/auto_examples/cluster/plot_agglomerative_dendrogram.html#sphx-glr-auto-examples-cluster-plot-agglomerative-dendrogram-py
def plot_dendrogram(model,type_of_data, **kwargs):
    # Create linkage matrix and then plot the dendrogram
    # create the counts of samples under each node
    counts = np.zeros(model.children_.shape[0])
    n_samples = len(model.labels_)
    for i, merge in enumerate(model.children_):
        current_count = 0
        for child_idx in merge:
            if child_idx < n_samples:
                current_count += 1  # leaf node
            else:
                current_count += counts[child_idx - n_samples]
        counts[i] = current_count

    linkage_matrix = np.column_stack([model.children_, model.distances_,
                                      counts]).astype(float)

    # Plot the corresponding dendrogram
    plt.figure(figsize=(10,5.5))
    dendrogram(linkage_matrix, **kwargs)
    plt.savefig(f"Dendrogram_{type_of_data}")
    return linkage_matrix

# Functions for dtw
def get_dtw_diff_matrix(df,cols:list):
    """
    From a list of series, compute a distance matrix by computing the 
    DTW distance of all pairwise combinations of series.
    """
    diff_matrix = {}
    cross = itertools.product(cols, cols)
    for (col1, col2) in cross:
        series1 = df[col1]
        series2 = df[col2]
        diff = dtw(
            series1, 
            series2,
            keep_internals=True, 
            step_pattern=rabinerJuangStepPattern(2, "c")
            )\
            .normalizedDistance
        diff_matrix[(col1, col2)] = [diff]
    return diff_matrix

def plot_dtw(df,series1:str, series2:str) -> None:
  dtw_df = dtw(df[series1],\
            df[series2],\
            keep_internals=True,
            step_pattern=rabinerJuangStepPattern(2, "c"))
  plt.figure(figsize=(10,5.5))
  dtw_df.plot(type="twoway",offset=5)
  plt.set_title=(f'{series1} and {series2}')
  plt.savefig(f"DTW_{series1}_{series1}")
  plt.show()

def plot_elbow_silhoutte_k_evaluation(name_of_data: str,data_array,max_clusters):
  range_n_clusters = range(2,max_clusters)
  elbow = []
  s_score = []
  for n_clusters in range_n_clusters:
    clusterer = KMeans(n_clusters = n_clusters, random_state=42,init='k-means++',max_iter=1000,n_init=1)
    cluster_labels = clusterer.fit_predict(data_array)
    # Average silhouette score
    silhouette_avg = silhouette_score(data_array, cluster_labels)
    s_score.append(silhouette_avg)
    # Average SSE"
    elbow.append(clusterer.inertia_) # Inertia: Sum of distances of samples to their closest cluster center
    
  fig = plt.figure(figsize=(10,5.5))
  fig.suptitle(f"K-means clusters for {name_of_data}", fontsize=16)
  fig.add_subplot(121)
  plt.plot(range_n_clusters, elbow,'b-',label=f'{name_of_data} SSE')
  plt.xlabel("Number of cluster")
  plt.ylabel("Sum of squared error(SSE)")
  plt.legend()
  
  fig.add_subplot(122)
  # plt.title("Covid-19 Rt values silhouette method results")
  plt.plot(range_n_clusters, s_score,'b-',label=f'{name_of_data} \n Silhouette Score')
  plt.xlabel("Number of cluster")
  plt.ylabel("Silhouette Score")
  plt.legend()
  plt.show()

def plot_kmeans_clusters(data_array,number_of_clusters,name_of_data:str):
  # computing K-Means with K = number_of_clusters
  centroids,_ = kmeans(data_array,number_of_clusters)
  # assign each sample to a cluster
  idx,_ = vq(data_array,centroids)
  # some plotting using numpy's logical indexing
  fig, ax = plt.subplots(figsize=(10,5.5))
  fig.suptitle(f"K-means clusters for {name_of_data}", fontsize=12)
  for cluster in range(number_of_clusters):
    colours = ['ob','ok','or','og','om','oc','oy']
    ax.plot(data_array[idx==cluster,0],data_array[idx==cluster,1],colours[cluster],label=f'cluster {cluster}')
    plt.legend()
  plt.savefig(f"{PATH}/images/k_means_{name_of_data}")
  plt.show()
  return idx

## Load data

In [ ]:
scaled_data_location = f'{PATH}/mean_scaled_products/'
files = os.listdir(scaled_data_location)
files[0]
yarn_fiber = pd.read_csv(f'{scaled_data_location}/{files[0]}',parse_dates=['year'],index_col='year')
remove_null_values(yarn_fiber)

path_to_min_features = f'{PATH}/min_feats_ts'
path_to_parameter_eff_features = f'{PATH}/efficient_parameters'
path_to_parameter_comp_features = f'{PATH}/comprehensive_parameters'
min_feats = os.listdir(path_to_min_features)
yarn_fiber_min_feats = pd.read_csv(f'{path_to_min_features}/{min_feats[0]}',index_col='id')
remove_null_values(yarn_fiber_min_feats)

# Exclude highly correlated features
exclude = [
          #  'export_val__kurtosis',
           'export_val__variance',
          #  'export_val__mean',
           'export_val__skewness',
          #  'export_val__standard_deviation',
          #  'export_val__median',
           'export_val__sum_values',
           'export_val__maximum',
          #  'export_val__minimum',
           'export_val__length'
           ]

min_feats_yarn = yarn_fiber_min_feats[yarn_fiber_min_feats\
                                      .columns[~yarn_fiber_min_feats\
                                               .columns\
                                               .isin(exclude)]]
print(check_outliers(min_feats_yarn))
yarn_fiber_min_feats = removing_outliers(min_feats_yarn)

#TODO: https://stats.stackexchange.com/questions/427327/simple-outlier-detection-for-time-series
min_feats_yarn = min_feats_yarn.div(min_feats_yarn.mean(axis=0),axis=1)
product_excl_countries = yarn_fiber_min_feats[yarn_fiber_min_feats.index.isin(random_countries)]
product_by_countries = min_feats_yarn[min_feats_yarn.index.isin(random_countries)]
from scipy.stats.mstats import winsorize
scaler = StandardScaler()
scaled_yarn_min_feats = scaler.fit_transform(yarn_fiber_min_feats)

# TODO: create function that loops through list columns to be included in  
X_scaled_transposed = preprocessing.scale(
    np.asarray([
                np.asarray(min_feats_yarn['export_val__minimum']),
                np.asarray(min_feats_yarn['export_val__mean']),
                # np.asarray(min_feats_yarn['export_val__length']),
                # np.asarray(min_feats_yarn['export_val__skewness']),
                np.asarray(min_feats_yarn['export_val__kurtosis']),
                np.asarray(min_feats_yarn['export_val__median']),
                # np.asarray(min_feats_yarn['export_val__mean']),
                np.asarray(min_feats_yarn['export_val__standard_deviation']),
                # np.asarray(min_feats_yarn['export_val__variance'])
                ])).T

## EDA and correlation analysis

In [ ]:
fig = plt.figure(figsize=(16,5.5))
fig.add_subplot(121)
sns.heatmap(min_feats_yarn.corr(),annot=True)
# plt.title("Correlation between time series features")
plt.savefig(f"{PATH}/images/correlation_stats_features")

fig.add_subplot(122)
plt.plot(product_excl_countries,label="Here")
# plt.title("Yarn fiber features for randomly picked \n countries excluding outliers")
plt.xticks(rotation=70)
plt.ylabel("Export value")
plt.legend(product_excl_countries.columns)
plt.show()

# Clustering mean scaled time series data

## Hiearchical clustering

In [ ]:
clf = AgglomerativeClustering(n_clusters=None, distance_threshold = 0).fit(yarn_fiber.T.values)

# plot the top five levels of the dendrogram
plt.figure(figsize = (10,5.5))
plt.title=('Raw data hierarchical Clustering Dendrogram')
linkage_matrix = plot_dendrogram(clf, "raw_data",p=5, color_threshold = 110,truncate_mode='level')
plt.savefig("raw_data_hierarchical")
plt.show()
# extract clusters from dendogram
clusters = fcluster(linkage_matrix, 100, criterion='distance')
# create a lookup table for series in a given cluster
yarn_fiber_clusters = yarn_fiber.T.reset_index()
yarn_fiber_clusters["cluster"] = clusters
yarn_fiber_clusters.rename(columns={'index':'country'},inplace=True)
yarn_fiber_clustered = yarn_fiber_clusters.set_index("cluster country".split())\
    .sort_index()

# cluster analysis
clusters = yarn_fiber_clusters.cluster.unique()
print(clusters)
for c in clusters:
  countries= yarn_fiber_clustered.loc[c].index.get_level_values(0).unique()
  # random.seed(1)
  n_samples = yarn_fiber_clustered.loc[c].shape[0]
  if n_samples > 10:
    n = random.sample(range(n_samples),10)
  else:
    n = range(n_samples)
  cluster = yarn_fiber_clustered.loc[c].T
  cluster.iloc[:, n].plot(subplots=False,figsize = (10,5.5),title=f"cluster_{c}")
  plt.legend(countries)
  # plt.savefig(f"{PATH}/images/raw_data_cluster{c}")
  plt.show()

## Dynamic time warping

In [ ]:
# sample 50 series, and compute the DTW distance matrix
random.seed(10)
sample_cols = random.sample(list(yarn_fiber.columns), 100)
dtw_diff_dict = get_dtw_diff_matrix(yarn_fiber,sample_cols)

# make into a df
dtw_diff_df = pd.DataFrame(dtw_diff_dict).T.reset_index()\
    .rename(columns = {"level_0":"First_variable", "level_1":"Second_variable", 0:"diff"})\
    .pivot_table(index = "First_variable", columns = "Second_variable", values = "diff")

# plot a similarity matrix, with a dendogram imposed
import seaborn as sns
sns.clustermap(1-dtw_diff_df)

# ward clustering from difference matrix, where distance is Dynamic time warping distance instead of Euclidean
time_warp = ward(dtw_diff_df)
# extract clusters
dtw_clusters = pd.DataFrame({"cluster":fcluster(time_warp, 1.15)}, index = dtw_diff_df.index)
dtw_clusters.cluster.value_counts().sort_index().plot.barh()
plt.title=("Frequency of DTW clusters")

# Check time series for any cluster
# TODO: Function to loop through all clusters and plot
# What cluster is South Africa? 
#print(dtw_clusters[dtw_clusters.index=='South Africa'])
cluster = 1
yarn_hc_clusters = yarn_fiber.T.merge(
    dtw_clusters.loc[dtw_clusters.cluster ==cluster], 
    left_index = True,
    right_index = True)\
    .T
yarn_hc_clusters.plot(subplots=True,figsize = (10,5.5),sharey=True,title=f'Countries in cluster {cluster}')
plt.show()

print('DTW for Turkey and India')
plot_dtw(yarn_fiber,"Turkey", "India")

print('DTW for Rwanda and Montenegro')
plot_dtw(yarn_fiber,"Rwanda", "Montenegro")

print('DTW for Niger and Republic of the Congo')
plot_dtw(yarn_fiber,"Niger", "Republic of the Congo")

# Clustering TSFRESH extracted features

## Hierarchical clustering TS features

In [ ]:
feats_clf = AgglomerativeClustering(n_clusters=None, distance_threshold = 0).fit(min_feats_yarn.values)

plt.figure(figsize = (14,6))
# plot the top five levels of the dendrogram
linkage_matrix = plot_dendrogram(feats_clf,"min_feats_raw", p=3,color_threshold = 110,truncate_mode='level')
plt.show()

# extract clusters from dendogram
clusters = fcluster(linkage_matrix, 100, criterion='distance')
# create a lookup table for series in a given cluster
yarn_fiber_clusters = min_feats_yarn.reset_index()
yarn_fiber_clusters["cluster"] = clusters
clusts = yarn_fiber_clusters['cluster'].unique()
print(f'Unique clusters: {clusts}')
yarn_fiber_clusters.rename(columns={'id':'country'},inplace=True)
yarn_fiber_clustered = yarn_fiber_clusters.set_index("cluster country".split())\
.sort_index()


# cluster analysis
feats_clusters = yarn_fiber_clusters.cluster.unique()
for c in feats_clusters:
  countries= yarn_fiber_clustered.loc[c].index.get_level_values(0).unique()
  random.seed(1)
  n_samples = yarn_fiber_clustered.loc[c].shape[0]
  if n_samples > 10:
    n = random.sample(range(n_samples),10)
  else:
    n = range(n_samples)
  cluster = yarn_fiber_clustered.loc[c].T
  cluster.iloc[:, n].plot(subplots=True,figsize=(10,5.5),
                          title=f"Countries in cluster_{c}_min_feats")
  plt.xticks(rotation=70)
  plt.legend(countries)
  plt.savefig(f"{PATH}/images/cluster_{c}_min_feats")
  plt.show()

## K-means clustering

In [ ]:
plot_elbow_silhoutte_k_evaluation("yarn_scaled_min_feats",X_scaled_transposed,10)  
clusters_yarn_min_feats = plot_kmeans_clusters(X_scaled_transposed,6,"yarn_min_features") #TODO: seperate def get_clusters() \\ plot_kmeans_clusters()

#TODO: function for getting names from cluster
# def get_cluster_elements()
details = [(name,cluster) for name, cluster in zip(min_feats_yarn.index,clusters_yarn_min_feats)]
cluster_df = pd.DataFrame(details,columns=['names','cluster'])
cluster_df['names'].astype('category')
get_names = min_feats_yarn.reset_index().rename(columns={'id':'names'})
get_names.names.astype('category')
country_cluster = pd.merge(get_names,cluster_df,how='inner', on='names')
metrics = ['mean']
groups = country_cluster.groupby(['cluster']).agg(metrics)
groups.plot(figsize=(15,5.5),kind='bar')
country_cluster[country_cluster.cluster==2]['names'].unique()

## PCA + k-Means

In [ ]:
# Standardize the data to have a mean of ~0 and a variance of 1

# TODO: PCA plots and evaluation
# Create a PCA instance: pca
# def calculate_pca():
  # return pca_components_df

X_std = StandardScaler().fit_transform(min_feats_yarn)
pca = PCA(n_components=5)
principalComponents = pca.fit_transform(X_std)

# def plot_pca_eveluation():
# Plot the explained variances
features = range(pca.n_components_)

fig = plt.figure(figsize=(10,5.5))
fig.suptitle("Clusters for yarn min_features", fontsize=16)
fig.add_subplot(131)
plt.bar(features, pca.explained_variance_ratio_, color='black')
plt.xlabel('PCA features')
plt.ylabel('variance %')
plt.xticks(features)
# Save components to a DataFrame
PCA_components = pd.DataFrame(principalComponents)
fig.add_subplot(132)
plt.scatter(PCA_components[0], PCA_components[1], alpha=.4, color='green')
plt.xlabel('PCA 1')
plt.ylabel('PCA 2')


plot_elbow_silhoutte_k_evaluation("yarn_scaled_pca_min_feats",np.asarray(PCA_components),10)  
clusters_yarn_min_feats = plot_kmeans_clusters(np.asarray(PCA_components),5,"yarn_min_features_pca")

details = [(name,cluster) for name, cluster in zip(min_feats_yarn.index,clusters_yarn_min_feats)]
cluster_df = pd.DataFrame(details,columns=['names','cluster'])
cluster_df['names'].astype('category')
get_names = min_feats_yarn.reset_index().rename(columns={'id':'names'})
get_names.names.astype('category')
country_cluster = pd.merge(get_names,cluster_df,how='inner', on='names')
metrics = ['mean']
groups = country_cluster.groupby(['cluster']).agg(metrics)
groups.plot(figsize=(15,5.5),kind='bar')
plt.savefig('clusters_yarn_min_features_pca_kmeans')

## DBSCAN

### Hyperparameter tuning

In [ ]:
# https://stackoverflow.com/questions/34611038/grid-search-for-hyperparameter-evaluation-of-clustering-in-scikit-learn
!pip install -q clusteval
# Import library
from clusteval import clusteval
# Set parameters, as an example dbscan
ce = clusteval(method='dbscan')
# Fit to find optimal number of clusters using dbscan
results= ce.fit(X_scaled_transposed)
# Make plot of the cluster evaluation
plt.figure(figsize=(10,5.5))
ce.plot()
# Make scatter plot. Note that the first two coordinates are used for plotting.
ce.scatter(X_scaled_transposed)
# results is a dict with various output statistics. One of them are the labels.
cluster_labels = results['labx']

### DBSCAN clustering

In [2]:
# https://www.dummies.com/programming/big-data/data-science/how-to-create-an-unsupervised-learning-model-with-dbscan/
dbscan = DBSCAN(eps=0.5, min_samples=5,metric='euclidean')
dbscan_results_yarn = dbscan.fit(X_scaled_transposed)
print(np.unique(dbscan.labels_))

NameError: name 'DBSCAN' is not defined

In [ ]:
# https://www.dummies.com/programming/big-data/data-science/how-to-create-an-unsupervised-learning-model-with-dbscan/
dbscan = DBSCAN(eps=0.5, min_samples=5,metric='euclidean')
dbscan.fit(X_scaled_transposed)
fig = plt.figure(figsize=(10,5.5))
fig.suptitle("Clusters for PCA-DBSCAN : yarn min_features", fontsize=16)
print(np.unique(dbscan.labels_))
fig.add_subplot(121)
# fig.set_title('Clusters')
pca = PCA(n_components=2)
pca_2d = pca.fit_transform(X_scaled_transposed)
for i in range(0, pca_2d.shape[0]):
  if dbscan.labels_[i] == 0:
    c1 = plt.scatter(pca_2d[i,0],pca_2d[i,1],c='r',marker='+')
  elif dbscan.labels_[i] == 1:
    c2 = plt.scatter(pca_2d[i,0],pca_2d[i,1],c='g',marker='o')
  elif dbscan.labels_[i] == 2:
    c3 = plt.scatter(pca_2d[i,0],pca_2d[i,1],c='b',marker='*')
  elif dbscan.labels_[i] == 3:
    c4 = plt.scatter(pca_2d[i,0],pca_2d[i,1],c='y',marker='x')
  elif dbscan.labels_[i] == 4:
    c5 = plt.scatter(pca_2d[i,0],pca_2d[i,1],c='g',marker='*')
  elif dbscan.labels_[i] == 5:
    c6 = plt.scatter(pca_2d[i,0],pca_2d[i,1],c='r',marker='x')
  elif dbscan.labels_[i] == -1:
    c7 = plt.scatter(pca_2d[i,0],pca_2d[i,1],c='c',marker='+')
plt.xlabel("Number of clusters_dbscan")
plt.legend([c1, c2, c3,c4,c5,c6,c7], ['Cluster 1', 
                                'Cluster 2',
                                'Cluster 3',
                                'Cluster 4',
                                'Cluster 5',
                                'Cluster 6',
                                'Noise'])

# plt.title('DBSCAN finds 2 clusters and noise')
fig.add_subplot(122)
# plt.set_title=("Cluster instances/frequency")
plt.hist(dbscan.labels_,bins=8)
plt.savefig(f"{PATH}/images/yarn_min_feats_pca_dbscan")
plt.show()